- embeddings
    - 不同的向量模型, 会导致精确度吗
        - prompt 和 vector database 的模型要保持一致
    - 好处
    - 不好的地方, 召回失败
    - https://www.bilibili.com/video/BV1Q3NUetEzV
    - https://www.bilibili.com/video/BV1NANLeREUe

In [9]:
import openai
import os
from dotenv import load_dotenv

load_dotenv(override=True)
client = openai.OpenAI(api_key=os.getenv("OPENAI_API_KEY"), base_url=os.getenv("OPENAI_BASE_URL"))
model = os.getenv("OPENAI_MODEL")
# print(client.base_url)

In [8]:
import pandas as pd
import numpy as np
from ast import literal_eval

datafile_path = "data/fine_food_reviews_with_embeddings_1k.csv"

df = pd.read_csv(datafile_path)
df["embedding"] = df.embedding.apply(literal_eval).apply(np.array)

In [16]:
from utils.embeddings_utils import get_embedding, cosine_similarity

# search through the reviews for a specific product
def search_reviews(df, product_description, n=3, pprint=True):
    product_embedding = get_embedding(
        product_description,
        model="text-embedding-3-small"
    )
    df["similarity"] = df.embedding.apply(lambda x: cosine_similarity(x, product_embedding))
    
    top_n = df.sort_values("similarity", ascending=False).head(n)
    top_n["combined"] = (
        top_n.combined
        .str.replace("Title: ", "")
        .str.replace("; Content:", ": ")
    )

    if pprint:
        for _, row in top_n.iterrows():
            print(f"Similarity: {row['similarity']:.4f}")
            print(row["combined"][:200])
            print()

    return top_n[["combined", "similarity"]]

results = search_reviews(df, "delicious beans", n=3, pprint=False)
print(results)

                                              combined  similarity
264  Delicious!:  I enjoy this white beans seasonin...    0.573562
647  Fantastic Instant Refried beans:  Fantastic In...    0.558107
296  Delicious:  While there may be better coffee b...    0.509843


In [18]:
results = search_reviews(df, "whole wheat pasta", n=3, pprint=False)
print(results)

                                              combined  similarity
325  Tasty and Quick Pasta:  Barilla Whole Grain Fu...    0.491543
474  sooo good:  tastes so good. Worth the money. M...    0.489431
29   Bland and vaguely gamy tasting, skip this one:...    0.444173


不同的单词, embedding 后, 最后求得相似度结果可能是一样

Different models utilize RAG: a good base model is important

ReRank